# October 31 - November 1 night time jet

In [15]:
import sys
sys.path.append('../')
import soslib
import xarray as xr
import altair as alt
import datetime
alt.data_transformers.disable_max_rows()
import pandas as pd 
import numpy as np
import glob

# Download SoS data

In [16]:
f1_sos = soslib.download_sos_data_day(date = '20221022', local_download_dir='sosnoqc')
f2_sos = soslib.download_sos_data_day(date = '20221028', local_download_dir='sosnoqc')
sos_data = xr.concat([
        xr.open_dataset(f1_sos),
        xr.open_dataset(f2_sos)
    ],
    dim='time'
)

# Identify variables we will use

In [17]:
VARIABLE_NAMES = [
    'w_h2o__1m_d', # moisture flux (AKA latent heat flux), m^3/m^2/s
    'w_h2o__3m_d',
    'w_h2o__10m_d',
     
    'w_tc__1m_d', # sensible heat flux, K/m^2/s
    'w_tc__3m_d',
    'w_tc__10m_d',

    'Rsw_in_9m_d', # incoming shortwave radiation, W/m^2
    'Rsw_out_9m_d',


    'Tsoil_3_1cm_d', # soil temp    
    'Tsoil_8_1cm_d',
    'Tsoil_18_1cm_d',
    'Tsoil_28_1cm_d',

    'Tsoil_4_4cm_d',
    'Tsoil_9_4cm_d',
    'Tsoil_19_4cm_d',
    'Tsoil_29_4cm_d',

    'Tsoil_0_6cm_d', 
    'Tsoil_10_6cm_d',
    'Tsoil_20_6cm_d',
    'Tsoil_30_6cm_d',

    'Tsoil_1_9cm_d',
    'Tsoil_11_9cm_d',
    'Tsoil_21_9cm_d',
    'Tsoil_31_9cm_d',

    'Gsoil_d',  # heat flux at depth, W/m^2
    'Qsoil_d',   # soil moisture, % volume

    'T_1m_c'
]

## Extract 5 minute-average data from the raw SoS dataset

In [18]:
sos_data_df = sos_data[VARIABLE_NAMES].to_dataframe().reset_index().melt(id_vars='time', value_vars=VARIABLE_NAMES)
sos_data_df['height'] = sos_data_df['variable'].apply(soslib.height_from_variable_name)
sos_data_df['tower'] = sos_data_df['variable'].apply(soslib.tower_from_variable_name)
sos_data_df['measurement'] = sos_data_df['variable'].apply(soslib.measurement_from_variable_name)
# sos_data_df['time'] = sos_data_df['time'] - dt.timedelta(hours=6)

# Examine dirunal cycles on different days

```
Can you consolidate and make a plot with 24-hr radiation balance and water vapor 
fluxes and sensible heat fluxes and soil temperature and soil water content, 
with different line styles for the two days? Thank you!
```

In [19]:
# Add net shortwave radiation rows
net_sw_data = pd.merge(
    sos_data_df.query('variable == "Rsw_out_9m_d"'),
    sos_data_df.query('variable == "Rsw_in_9m_d"'),
    on = ['time', 'height', 'tower']
)[['time', 'height', 'tower', 'value_x', 'value_y']]

net_sw_data['measurement'] = 'shortwave radiation net'
net_sw_data['variable'] = 'Rsw_net_9m_d'
net_sw_data['value'] = net_sw_data['value_y'] - net_sw_data['value_x']

sos_data_df = pd.concat([
    sos_data_df,
    net_sw_data.drop(columns=['value_x', 'value_y'])
])

In [20]:
# Add time columns for easy plotting
# the individual day, as a string
sos_data_df['day'] = sos_data_df['time'].dt.strftime('%m/%d/%Y')
# the time of the day, assuming a random date
sos_data_df['time_of_day'] = sos_data_df['time'].apply(lambda dt: datetime.datetime(2022, 1, 1, dt.hour, dt.minute, dt.second))


In [21]:
soil_temp_data = sos_data_df.query('measurement == "soil temperature"')
soil_temp_data = soil_temp_data.set_index(['time'])
soil_temp_data
soil_temp_data_1hr = soil_temp_data.groupby(['height', 'variable', 'tower', 'measurement', 'day']).resample('120Min').mean().drop(columns='height')
soil_temp_data_1hr = soil_temp_data_1hr.reset_index()
# Add time columns for easy plotting
# the individual day, as a string
soil_temp_data_1hr['day'] = soil_temp_data_1hr['time'].dt.strftime('%m/%d/%Y')
# the time of the day, assuming a random date
soil_temp_data_1hr['time_of_day'] = soil_temp_data_1hr['time'].apply(lambda dt: datetime.datetime(2022, 1, 1, dt.hour, dt.minute, dt.second))
soil_temp_data_1hr['hour'] = soil_temp_data_1hr['time'].dt.hour
soil_temp_data_1hr

/var/folders/jy/xv6qnlms2cj_3p5yrzcwc0n00000gn/T/ipykernel_32727/3215924086.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  soil_temp_data_1hr = soil_temp_data.groupby(['height', 'variable', 'tower', 'measurement', 'day']).resample('120Min').mean().drop(columns='height')


,height,variable,tower,measurement,day,time,value,time_of_day,hour
0,-0.319,Tsoil_31_9cm_d,d,soil temperature,10/22/2022,2022-10-22 00:00:00,7.713065,2022-01-01 00:00:00,0
1,-0.319,Tsoil_31_9cm_d,d,soil temperature,10/22/2022,2022-10-22 02:00:00,7.794468,2022-01-01 02:00:00,2
2,-0.319,Tsoil_31_9cm_d,d,soil temperature,10/22/2022,2022-10-22 04:00:00,7.897930,2022-01-01 04:00:00,4
3,-0.319,Tsoil_31_9cm_d,d,soil temperature,10/22/2022,2022-10-22 06:00:00,8.005783,2022-01-01 06:00:00,6
4,-0.319,Tsoil_31_9cm_d,d,soil temperature,10/22/2022,2022-10-22 08:00:00,8.075860,2022-01-01 08:00:00,8
...,...,...,...,...,...,...,...,...,...
379,-0.006,Tsoil_0_6cm_d,d,soil temperature,10/28/2022,2022-10-28 14:00:00,0.369109,2022-01-01 14:00:00,14
380,-0.006,Tsoil_0_6cm_d,d,soil temperature,10/28/2022,2022-10-28 16:00:00,0.325716,2022-01-01 16:00:00,16
381,-0.006,Tsoil_0_6cm_d,d,soil temperature,10/28/2022,2022-10-28 18:00:00,0.345021,2022-01-01 18:00:00,18
382,-0.006,Tsoil_0_6cm_d,d,soil temperature,10/28/2022,2022-10-28 20:00:00,0.454081,2022-01-01 20:00:00,20


In [30]:
alt.Chart(sos_data_df).transform_filter(
    alt.FieldOneOfPredicate(
        'variable', 
        ['T_1m_c', 'Rsw_net_9m_d', 'w_h2o__1m_d', 'w_tc__1m_d', 'Qsoil_d']
    )
).mark_line().encode(
    alt.X('time_of_day:T'),
    alt.Y('value:Q'),
    alt.Color('day:N')
).properties(
    width=1000, 
    height = 150
).facet(
    row='variable'
).resolve_scale(y='independent', x='shared') & \
alt.Chart(soil_temp_data_1hr).mark_line().encode(
    alt.X('value:Q', sort='y', ),
    alt.Y('height:Q', axis=alt.Axis(grid=False)),
    alt.Color('day:N')
).properties(
    width=73,
    height = 100
).facet(
    # row='day:N',
    column='hour:O',
    spacing=7
)

alt.VConcatChart(...)